In [2]:
# General libraries:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Load historical data from file:
df = pd.read_csv('eth_historical_4hrs.csv')

# Convert timezones and reformat:
df['time_period_start'] = pd.to_datetime(df['time_period_start']).dt.tz_convert('US/Central').dt.strftime('%m-%d-%Y %H:%M')

# Pull just the value fields:
df = df[['price_close', 'price_open', 'price_high', 'price_low', 'volume_traded', 'trades_count']]

# Calculate MACD:
df['26_ema'] = df['price_close'].ewm(span=26,min_periods=0,adjust=True,ignore_na=False).mean()
df['12_ema'] = df['price_close'].ewm(span=12,min_periods=0,adjust=True,ignore_na=False).mean()
df['MACD'] = df['12_ema'] - df['26_ema']

df.head()

,price_close,price_open,price_high,price_low,volume_traded,trades_count,26_ema,12_ema,MACD
0,11.8895,11.87000,11.90000,11.85972,1076.171006,48,11.889500,11.889500,0.000000
1,12.0000,11.89000,12.00000,11.89000,6184.443260,182,11.946875,11.949354,0.002479
2,12.0300,12.00000,12.03000,11.93356,3391.487460,120,11.976741,11.980830,0.004089
3,12.2000,12.02591,12.20000,12.00000,1590.240506,53,12.039154,12.050014,0.010859
4,12.4310,12.20000,12.54799,12.20000,4801.330298,164,12.130025,12.153526,0.023501


In [9]:
def entry_flag(df, lookforward_num=6, pct_gain=0.03):
    
    '''If there will be over a x% gain in the next x periods, flag as a 1. Else 0.'''
    
    df['pctchange'] = df[['price_close']].pct_change(lookforward_num)
    
    entry_flags = []
    for i in range(len(df) - lookforward_num):
        if df.iloc[i+lookforward_num]['pctchange'] >= pct_gain:
            entry_flags.append(1)
        else:
            entry_flags.append(0)
    # Add in buffer None's
    for i in range(lookforward_num):
        entry_flags.append(None)
    return entry_flags

In [10]:
lookforward_num = 6
pct_gain=0.05
df['entry_flag'] = entry_flag(df, lookforward_num, pct_gain)
df['entry_flag'].value_counts()

0.0    7201
1.0    1206
Name: entry_flag, dtype: int64

In [16]:
df.iloc[1000:1050]

,price_close,price_open,price_high,price_low,volume_traded,trades_count,26_ema,12_ema,MACD,pctchange,entry_flag
1000,45.25999,44.73000,45.25999,44.55380,1994.056755,356,44.652926,44.108050,-0.544876,0.052584,0.0
1001,44.84739,45.28001,45.62303,44.70102,3761.893691,282,44.667331,44.221795,-0.445536,0.073418,0.0
1002,44.33113,44.84811,45.10379,44.33110,1222.865626,278,44.642427,44.238616,-0.403812,0.049506,0.0
1003,43.81057,44.30502,44.43200,43.53012,2602.592017,415,44.580808,44.172763,-0.408046,-0.004380,0.0
1004,44.57783,43.80071,44.65191,43.51636,1158.154632,169,44.580588,44.235081,-0.345507,0.003461,0.0
1005,43.51092,44.60830,44.99963,43.33308,3327.241739,257,44.501353,44.123671,-0.377682,-0.027254,0.0
1006,43.80000,43.67570,44.00000,43.21000,9407.228465,688,44.449401,44.073876,-0.375525,-0.032258,0.0
1007,43.43600,43.43264,44.26978,43.21000,2876.832544,361,44.374334,43.975741,-0.398593,-0.031471,0.0
1008,43.93521,43.73670,44.18819,43.64989,398.819125,147,44.341807,43.969505,-0.372301,-0.008931,0.0
1009,43.80004,43.80009,44.12690,43.60000,708.820356,208,44.301676,43.943434,-0.358242,-0.000240,0.0
